# I. $LU$ - разложение квадратной матрицы

Рассмотрим наивную реализацию LU - разложения.  

Заметим, что мы используем массивы `numpy` для представления матриц. [Не используйте 'np.matrix'].

In [1]:
import numpy as np

def diy_lu(a):
    """Создает LU - разложение матрицы `a`.
    
    Наивное LU - разложение: работает столбец за столбцом, накапливает элементарные треугольные матрицы.
    Без выбора главного элемента.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Теперь сгенерируем матрицу полного ранга и протестируем наивное разложение.
import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Настройка вывода чисел с плавающей точкой для большей ясности
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(a, "\n")
print(L, "\n")
print(u, "\n")

# Быстрый тест на адекватность: L @ U должна быть равна изначальной матрице с точностью до ошибок округления.
print(L@u - a)

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.00

# II. Необходимость выбора главного элемента

Давайте немного подправим матрицу, изменив в ней один элемент:

In [5]:
a1 = a.copy()
a1[1, 1] = 3

Результирующая матрица имеет полный ранг, но наивное LU - разложение не работает.

In [6]:
np.linalg.matrix_rank(a1)

6

In [34]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


<ipython-input-1-8159eb3fe7f6>:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-1-8159eb3fe7f6>:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-1-8159eb3fe7f6>:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-1-8159eb3fe7f6>:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]


### Тест II.1

Для того, чтобы наивное LU - разложение работало необходимо чтобы все лидирующие миноры матрицы были отличны от нуля. Проверьте, выполнено ли это требование для двух матриц `a` и `a1`.

(20% оценки)

In [35]:
def checker(A):
    n = A.shape[0]
    
    for i in range(1, n):
        print(i, abs(np.linalg.det(A[i:,i:])), '\n')
        if abs(np.linalg.det(A[i:,i:])) < 1e-16:
            return False
    return True

print(checker(a), "\n")
print(checker(a1), "\n")

1 7.419551442783735e-14 

2 3.7203871395457716e-12 

3 1.891286294340854e-08 

4 0.00018840013397343028 

5 0.1875 

True 

1 4.259631046811274e-12 

2 3.7203871395457716e-12 

3 1.891286294340854e-08 

4 0.00018840013397343028 

5 0.1875 

True 



### Тест II.2

Модифицируйте алгоритм `diy_lu`, чтобы осуществлять выбор главного элемента в столбцах. Для контроля выбора можете использовать матрицу перестановок или массив замен.

(40% оценки)

Напишите функию, воссоздающую изначальную матрицу из разложения. Протестируйте свой алгоритм на матрицах `a` и `a1`.

(40% оценки)

In [49]:
def P_columns(i, j):
    P= np.zeros((N, N), dtype=int)
    for k in range(N):
        if k != i and k != j:
            P[k][k] = 1
        elif k == i:
            P[i][j] = 1
        else:
            P[j][i] = 1
    return P

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.071 1.364 1.875 0.882 0.75 ]
 [3.    0.652 0.882 1.364 0.517 0.429]
 [3.    0.469 0.652 1.071 0.366 0.3  ]
 [3.    0.366 0.517 0.882 0.283 0.231]
 [3.    0.3   0.429 0.75  0.231 0.188]]


In [98]:
def make_a(a):
    max_columns = []
    columns = []
    max_i = [0, 0]
    for i in range(a.shape[0]-1, -1, -1):
        for j in range(a.shape[0] - i, a.shape[0]):
            if max_i[0] < abs(a[j][i]):
                max_i[0] = abs(a[j][i])
                max_i[1] = j
        print(i, max_i[1])
        a = np.dot(a, P_columns(i, max_i[1]))
        max_i[0] = 0
    return a

print(a1, '\n')
print(make_a(a1))

[[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 

5 1
4 2
3 3
2 4
1 5
0 5
[[3.    3.    3.    3.    3.    3.   ]
 [0.75  3.    1.364 1.071 0.882 3.   ]
 [0.429 1.364 0.882 0.652 0.517 3.   ]
 [0.3   1.071 0.652 0.469 0.366 3.   ]
 [0.231 0.882 0.517 0.366 0.283 3.   ]
 [0.188 0.75  0.429 0.3   0.231 3.   ]]


In [99]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


<ipython-input-1-8159eb3fe7f6>:15: RuntimeWarning: divide by zero encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
<ipython-input-1-8159eb3fe7f6>:17: RuntimeWarning: invalid value encountered in matmul
  u = lam @ u
<ipython-input-1-8159eb3fe7f6>:20: RuntimeWarning: invalid value encountered in matmul
  L = L @ lam
<ipython-input-1-8159eb3fe7f6>:15: RuntimeWarning: invalid value encountered in true_divide
  gamma = u[j+1:, j] / u[j, j]
